# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [1]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, \
                     SDS_tools, SDS_transects, NOC_download, NOC_shoreline


## 1. Retrieval of the images from GEE

To retrieve from the GEE server the available satellite images cropped around the user-defined region of coastline for the particular time period of interest, the following variables are required:

* Coordinate list: a list of the coordinates of the region of interest (longitude/latitude pairs in WGS84) – see section 'create coordinate list in User handbook to extract ROI from study site
* all_dates: dates over which the images will be retrieved (e.g., dates = ['2017-12-01', '2018-01-01'])
* all_sats: satellite missions to consider (e.g., sat_list = ['L7', 'L8', 'S2'] for Landsat 7, 8 and Sentinel-2 collections)
* sitename: name of the site (this is the name of the subfolder where the images and other accompanying files will be stored)

Similar to coastsat, there are some extra parameters such as those to go through manual checking, whether to combine Landsat collections and cloud thresholds. Some parameters have remained, such as 'cloud_mask_issue', but will not need changing to true as median images are often cloud free.

Make sure the area of your ROI is smaller than 100 km2 (if larger split it into smaller ROIs) - GEE limits download size

In [2]:
coordinate_list =([[151.35020966,-33.62163388],[151.35020966,-33.70486004],[151.30212338,-33.62163388],[151.30212338,-33.70486004]]),\
([[151.3223859,-33.69740968],[151.3223859,-33.80488455],[151.28327117,-33.69740968],[151.28327117,-33.80488455]])


#IMPORTANT - The code will retrieve the satellite data from the the defined time period and satellite
#            dependant on the number in the list. I.e. ['2000-01-01', '2000-12-31'] or all_dates[0]
#            corresponds to ['L7'] or all_sats[0]. 

all_dates = ([['2014-01-01', '2014-12-31'],['2016-01-01', '2016-12-31'],['2018-01-01', '2018-12-31'],['2020-01-01', '2020-12-31']])

all_sats = ([['L7'],['L8'],['S2'],['S2']])

study_area = 'NARRA'

# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')

# Settings List
settings = {'output_epsg': 32754,        # EPSG code (can be found here https://epsg.io/)
           'cloud_mask_issue':False,    # if there is an issue with the cloud mask and sand pixels
                                        # are erroneously being masked on the images
           'check_detection': False,     # lets user manually accept/reject the mapped shorelines
           'save_figure': False,         # saves a -jpg file for each mapped shoreline
           'adjust_detection': False,    # allows user to manually adjust the detected shoreline
           
            # [FOR ADVANCED USERS] shoreline detection parameters:
           'min_beach_area': 1000,     # minimum area (in metres^2) for an object to be labelled as a beach
           'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
           'min_length_sl': 0,       # minimum length (in metres) of shoreline perimeter to be valid
           'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
           'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
            
           #Co-registration
           'coregistration': False,    # Leave paramter 'false'. Current method not suffcient
            
           ## Image download Parameters
           # Landsat
           'LCloudScore': 15,         # Mean cloud score threshold (include images with less then threshold)
           'add_L7_to_L5': True,      # Add Landsat 7 to Landsat 5 median composite if they are in same time period
           'add_L5_to_L7': True,      # Add Landsat 5 to Landsat 7 median composite if they are in same time period
           'add_L7_to_L8': True,      # Add Landsat 7 to Landsat 8 median composite if they are in same time period
           'LCloudThreshold': 35,     # Pixels from a single image in a collection larger than this cloud score threshold
                                      # will be masked.
           # Sentinel
           'CLOUD_FILTER': 50,          # [Integer] Maximum image cloud cover percent allowed in image collection'
           'CLD_PRB_THRESH': 25,        # {Integer] Cloud probability (%); values greater than are considered cloud
           'NIR_DRK_THRESH': 0.08,      # [Float] Near-infrared reflectance; values less than are considered potential cloud shadow
           'CLD_PRJ_DIST': 2,           # [Float] Maximum distance (km) to search for cloud shadows from cloud edges |
           'BUFFER': 50,               # [Integer] Distance (m) to dilate the edge of cloud-identified objects |
            
        }

download = True

In [4]:
if download:
    
    for site in coordinate_list:       
             
        counter = 0    
        for date in all_dates:
            
            sat = all_sats[counter]
            settings['dates'] = date

            sitename = study_area + str(coordinate_list.index(site))

            # put all the inputs into a dictionnary
            inputs = {'polygon': site, 'dates': date, 'sat_list': sat,
              'sitename': sitename, 'filepath': filepath,
              'include_T2': False}

            #printProgress(f'processing {sitefolder}: {date}')
            NOC_download.retrieve_images(settings, inputs)
            
            counter = counter + 1
    print('FINISHED ...')

Images available between 2014-01-01 and 2014-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 18 images

- Cloud minimal images in Median:
   L7: 12
   L5: 0
   Total: 12
Median Processed
Downloaded

Images available between 2016-01-01 and 2016-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L8: 39 images

- Cloud minimal images in Median:
   L8: 20
   L7: 13
   Total: 33
Median Processed
Downloaded

Images available between 2018-01-01 and 2018-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 140 images

- Cloud minimal images in Median:
   S2: 96
Median processed


URLError: <urlopen error [Errno 11002] getaddrinfo failed>

In [ ]:
##Batch shoreline detection
for site in coordinate_list:       

    for sat in all_sats:

        for date in all_dates:

            settings['dates'] = date

            sitename = study_area + str(coordinate_list.index(site))

            # put all the inputs into a dictionnary
            inputs = {'polygon': site, 'dates': date, 'sat_list': sat,
              'sitename': sitename, 'filepath': filepath,
              'include_T2': False}
         
            settings.update({'inputs': inputs})
            
    %matplotlib qt            
    metadata = NOC_download.get_metadata(inputs)
    output = NOC_shoreline.extract_shorelines(metadata, settings, inputs)
            
print('Finished...')